In this notebook I will show you how to map helmet bbox and NGS tracking data.

- I used [train|test]_baseline_helmets.csv for helmet bbox.
- Compare the NGS tracking data with the lateral coordinates of the boxes.
- Map the players to have the smallest difference between each boxes and coordinates.
- The reason why I did not use vertical coordinates is because simplicity and the lateral direction is more accurate, but we can get more accurate mapping if we use depth as well.
- The direction of the axis changes depending on the camera position, so I choose the one that gives smaller distance for both directions.
- Since the camera and the XY axis will be rotated if it is out of the center of the court. So the angle at which the distance is the smallest is selected by grid-search.

Please check the code for details.

In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random

## setting and loading data

In [ ]:
debug = False
CONF_THRE = 0.3
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

## adding Estimated video frame to tracking

I used @robikscube's great code here.

In [ ]:
# copied from https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    tracks["time"] = pd.to_datetime(tracks["time"])
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype("timedelta64[ms]") / 1_000
    # Estimated video frame
    tracks["est_frame"] = (((tracks["snap_offset"] * fps) + snap_frame).round().astype("int"))
    return tracks
tracking = add_track_features(tracking)

In [ ]:
if debug:
    sample_keys = random.sample(list(tracking['gameKey'].unique()), 3)
    helmets['gameKey'] = helmets['video_frame'].str.split('_').str[0]
    tracking = tracking[tracking['gameKey'].isin(sample_keys)]
    helmets = helmets[helmets['gameKey'].astype(int).isin(sample_keys)]
    labels = labels[labels['gameKey'].astype(int).isin(sample_keys)]
tracking.shape, helmets.shape, labels.shape

## mapping helmet box and NGS tracking data

In [ ]:
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def norm_arr(a):
    a = a-a.min()
    a = a/a.max()
    return a
    
def dist(a1, a2):
    return np.linalg.norm(a1-a2)

max_iter = 2000
def dist_for_different_len(a1, a2):
    """
    Compute the L2 distance bewteen a1 and a2, 
    ================================================================================================================
    Inputs
        a1: x-axis values of players tracking sensor reading, after rotation.
        a2: x-axis values of helmet boxes on image.
    ---------------------------------------------------------------------------------------------------------------
    Return
        A tuple of two elements. 
        The 1st element is a scalar of the L2 distance between a1 and a2 (after both being normalised)
        The 2nd element is indices of a1 where no good matches are found in a2.
            
        ###########################################################################
        What does he do when len(a1) > len(a2)?
            - He wants to remove some items from a1, so that it has the same size as a2.
            - he needs to choose which items to be removed.
            - So try all possible combinations of removables! (except when too many items)
            - Find the case where L2 distance is minimised, with the corresponding removable indices in a1.
        ###########################################################################
    
    ================================================================================================================
    """
    assert len(a1) >= len(a2), f'{len(a1)}, {len(a2)}'   # make sure tracking sensor has more men than in the image.
    len_diff = len(a1) - len(a2)
    a2 = norm_arr(a2)
    if len_diff == 0:
        a1 = norm_arr(a1)
        return dist(a1,a2), ()
    else:
        min_dist = 10000
        min_detete_idx = None
        cnt = 0        
        
        del_list = list(itertools.combinations(range(len(a1)),len_diff))   
        
        if len(del_list) > max_iter:
            del_list = random.sample(del_list, max_iter)
        for detete_idx in del_list:
            this_a1 = np.delete(a1, detete_idx)
            this_a1 = norm_arr(this_a1)
            this_dist = dist(this_a1, a2)
            if min_dist > this_dist:
                min_dist = this_dist
                min_detete_idx = detete_idx
        return min_dist, min_detete_idx
        
def rotate_arr(u, t, deg=True):
    if deg == True:
        t = np.deg2rad(t)
    R = np.array([[np.cos(t), -np.sin(t)],
                  [np.sin(t),  np.cos(t)]])
    return  np.dot(R, u)

def dist_rot(tracking_df, a2):
    """
    Find the players that have the best matches. The best matches are obtained when the L2 distance is minimised 
    between the helmet boxes and the concurrent tracking vectors.
    ================================================================================================================
    Inputs
        tracking_df: tracking sensor data. if the corresponding image is taken from endzone, then the tracking
                     sensor's (x,y) has already been swapped.
        a2: centers of the helmet boxs on the x-axis.
    ---------------------------------------------------------------------------------------------------------------
    Return
        A tuple of two elements. 
        The 1st element is a scalar of the L2 distance between a1 and a2 (after both being normalised)
        The 2nd element is a list of the palyers that constitute the good matches.
    ================================================================================================================
    """
    tracking_df = tracking_df.sort_values('x')
    x = tracking_df['x']
    y = tracking_df['y']
    min_dist = 10000
    min_idx = None
    min_x = None
    dig_step = 3
    dig_max = dig_step*10
    for dig in range(-dig_max,dig_max+1,dig_step):   # -30 to +30 degrees 
        arr = rotate_arr(np.array((x,y)), dig)
#         print(arr,'\n')
        this_dist, this_idx = dist_for_different_len(np.sort(arr[0]), a2)   # only keep the x-axis values for all players, after rotation?
        if min_dist > this_dist:
            min_dist = this_dist
            min_idx = this_idx
            min_x = arr[0]
    tracking_df['x_rot'] = min_x
    player_arr = tracking_df.sort_values('x_rot')['player'].values
    players = np.delete(player_arr,min_idx)
    return min_dist, players


def mapping_df(args):
    """
    Map helmet boxes on the image with tracking sensor data (players). These are some important properties:
        - Try different rotations of the image data.
        - Condense both the image data and the tracking data to 1D
            1) Always use the x-axis of the rotated images (sideline/endzone), this is referred to as "a1"
            2) For the tracking sensor, use x-axis when image is from sideline, and y-axis when image 
                is from endzone. Denoted by "a2".
        - Matches are based on L2 distances between the vectors a1 and a2.
        - For each time instance, the best match is the correspondence between each helmet box location in 
           image and the player associated with an index in a2.
        - Since the orientation of the image x-axis can be flipped (maybe from different sides of the field),
            so both directions are tried, denoted by a2_p and a2_m.
    """
    video_frame, df = args
    gameKey,playID,view,frame = video_frame.split('_')
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
    len_this_tracking = len(this_tracking)
    df['center_h_p'] = (df['left']+df['width']/2).astype(int)
    df['center_h_m'] = (df['left']+df['width']/2).astype(int)*-1
    df = df[df['conf']>CONF_THRE].copy()
    if len(df) > len_this_tracking:
        df = df.tail(len_this_tracking)
    df_p = df.sort_values('center_h_p').copy()
    df_m = df.sort_values('center_h_m').copy()
    
    if view == 'Endzone':
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
    a2_p = df_p['center_h_p'].values
    a2_m = df_m['center_h_m'].values

    ############################################################################################################
    # "min_detete_idx" is a misued of variable name here; it should be "min_included_idx"
    ############################################################################################################
    
    min_dist_p, min_detete_idx_p = dist_rot(this_tracking ,a2_p)   
    min_dist_m, min_detete_idx_m = dist_rot(this_tracking ,a2_m)   
    if min_dist_p < min_dist_m:
        min_dist = min_dist_p
        min_detete_idx = min_detete_idx_p
        tgt_df = df_p
    else:
        min_dist = min_dist_m
        min_detete_idx = min_detete_idx_m
        tgt_df = df_m
        
    tgt_df['label'] = min_detete_idx

    return tgt_df[['video_frame','left','width','top','height','label']]

## Jacky's demo

Based on instance `57906_000718_Sideline_159`

In [ ]:
# import glob
# glob.glob('/kaggle/input/nfl-health-and-safety-helmet-assignment/train/57906*')

import cv2
vidcap = cv2.VideoCapture('/kaggle/input/nfl-health-and-safety-helmet-assignment/train/57906_000718_Sideline.mp4')
vid = np.array([vidcap.read()[1] for i in range(160)])
print(vid.shape)

In [ ]:
plt.figure(figsize=(25,9))
plt.imshow(vid[159,:,:,:])
plt.title('The 160th frame', fontsize=30, color='blue')

In [ ]:
n = 500
df_list = list(helmets.groupby('video_frame'))
video_frame = df_list[n][0]
raw_df = df_list[n][1]
df = mapping_df(df_list[n])
print(video_frame)
gameKey,playID,view,frame = video_frame.split('_')
gameKey = int(gameKey)
playID = int(playID)
frame = int(frame)
this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
est_frame = find_nearest(this_tracking.est_frame.values, frame)
this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
this_tracking.head()

In [ ]:
# raw_df

In [ ]:
# for demonstration purpose, ignore rotation.

a1 = norm_arr(this_tracking['x']).sort_values().values
a2 = norm_arr(raw_df['center_h_p'])
print('Original sizes')
print(a1.shape, a2.shape)
print('Random sample - make a1 and a2 the same size')
score_dict = {}
for i in range(10):
    temp2 = a2.sample(len(a1)).sort_values()
    print(i,np.sum((a1-temp2.values)**2)**0.5)
    score_dict[i] = temp2.index

In [ ]:
# df.loc[score_dict[6],:]

In [ ]:
temp_x = df['left'] + df['width']/2
temp_y = df['top'] + df['height']/2

fig1, ax1 = plt.subplots(ncols=2,nrows=1,figsize=(20,8))
ax1[0].scatter(this_tracking['x'],this_tracking['y'])
ax1[0].set_title('tracking sensor')
ax1[0].set_ylim([53.3,0])
ax1[0].set_xlim([40, 0])
ax1[1].scatter(temp_x,temp_y)
ax1[1].set_ylim([720, 0])
ax1[1].set_title('image frame')
ax1[0].grid(True)
ax1[1].grid(True)

## The original author's work below: iterate all data

In [ ]:
# p = Pool(processes=4)
# submission_df_list = []
# df_list = list(helmets.groupby('video_frame'))
# with tqdm(total=len(df_list)) as pbar:
#     for this_df in p.imap(mapping_df, df_list):   # mapping_df is the function defined above
#         submission_df_list.append(this_df)
#         pbar.update(1)
# p.close()

In [ ]:
# submission_df = pd.concat(submission_df_list)
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
# import time
# i = 0
# while True:
#     time.sleep(1)
#     print(i, end='\r')
#     i += 1

## validation
I used @robikscube's great code here.

In [ ]:
# # copied from https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
# class NFLAssignmentScorer:
#     def __init__(
#         self,
#         labels_df: pd.DataFrame = None,
#         labels_csv="train_labels.csv",
#         check_constraints=True,
#         weight_col="isDefinitiveImpact",
#         impact_weight=1000,
#         iou_threshold=0.35,
#         remove_sideline=True,
#     ):
#         """
#         Helper class for grading submissions in the
#         2021 Kaggle Competition for helmet assignment.
#         Version 1.0
#         https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide

#         Use:
#         ```
#         scorer = NFLAssignmentScorer(labels)
#         scorer.score(submission_df)

#         or

#         scorer = NFLAssignmentScorer(labels_csv='labels.csv')
#         scorer.score(submission_df)
#         ```

#         Args:
#             labels_df (pd.DataFrame, optional):
#                 Dataframe containing theground truth label boxes.
#             labels_csv (str, optional): CSV of the ground truth label.
#             check_constraints (bool, optional): Tell the scorer if it
#                 should check the submission file to meet the competition
#                 constraints. Defaults to True.
#             weight_col (str, optional):
#                 Column in the labels DataFrame used to applying the scoring
#                 weight.
#             impact_weight (int, optional):
#                 The weight applied to impacts in the scoring metrics.
#                 Defaults to 1000.
#             iou_threshold (float, optional):
#                 The minimum IoU allowed to correctly pair a ground truth box
#                 with a label. Defaults to 0.35.
#             remove_sideline (bool, optional):
#                 Remove slideline players from the labels DataFrame
#                 before scoring.
#         """
#         if labels_df is None:
#             # Read label from CSV
#             if labels_csv is None:
#                 raise Exception("labels_df or labels_csv must be provided")
#             else:
#                 self.labels_df = pd.read_csv(labels_csv)
#         else:
#             self.labels_df = labels_df.copy()
#         if remove_sideline:
#             self.labels_df = (
#                 self.labels_df.query("isSidelinePlayer == False")
#                 .reset_index(drop=True)
#                 .copy()
#             )
#         self.impact_weight = impact_weight
#         self.check_constraints = check_constraints
#         self.weight_col = weight_col
#         self.iou_threshold = iou_threshold

#     def check_submission(self, sub):
#         """
#         Checks that the submission meets all the requirements.

#         1. No more than 22 Boxes per frame.
#         2. Only one label prediction per video/frame
#         3. No duplicate boxes per frame.

#         Args:
#             sub : submission dataframe.

#         Returns:
#             True -> Passed the tests
#             False -> Failed the test
#         """
#         # Maximum of 22 boxes per frame.
#         max_box_per_frame = sub.groupby(["video_frame"])["label"].count().max()
#         if max_box_per_frame > 22:
#             print("Has more than 22 boxes in a single frame")
#             return False
#         # Only one label allowed per frame.
#         has_duplicate_labels = sub[["video_frame", "label"]].duplicated().any()
#         if has_duplicate_labels:
#             print("Has duplicate labels")
#             return False
#         # Check for unique boxes
#         has_duplicate_boxes = (
#             sub[["video_frame", "left", "width", "top", "height"]].duplicated().any()
#         )
#         if has_duplicate_boxes:
#             print("Has duplicate boxes")
#             return False
#         return True

#     def add_xy(self, df):
#         """
#         Adds `x1`, `x2`, `y1`, and `y2` columns necessary for computing IoU.

#         Note - for pixel math, 0,0 is the top-left corner so box orientation
#         defined as right and down (height)
#         """

#         df["x1"] = df["left"]
#         df["x2"] = df["left"] + df["width"]
#         df["y1"] = df["top"]
#         df["y2"] = df["top"] + df["height"]
#         return df

#     def merge_sub_labels(self, sub, labels, weight_col="isDefinitiveImpact"):
#         """
#         Perform an outer join between submission and label.
#         Creates a `sub_label` dataframe which stores the matched label for each submission box.
#         Ground truth values are given the `_gt` suffix, submission values are given `_sub` suffix.
#         """
#         sub = sub.copy()
#         labels = labels.copy()

#         sub = self.add_xy(sub)
#         labels = self.add_xy(labels)

#         base_columns = [
#             "label",
#             "video_frame",
#             "x1",
#             "x2",
#             "y1",
#             "y2",
#             "left",
#             "width",
#             "top",
#             "height",
#         ]

#         sub_labels = sub[base_columns].merge(
#             labels[base_columns + [weight_col]],
#             on=["video_frame"],
#             how="right",
#             suffixes=("_sub", "_gt"),
#         )
#         return sub_labels

#     def get_iou_df(self, df):
#         """
#         This function computes the IOU of submission (sub)
#         bounding boxes against the ground truth boxes (gt).
#         """
#         df = df.copy()

#         # 1. get the coordinate of inters
#         df["ixmin"] = df[["x1_sub", "x1_gt"]].max(axis=1)
#         df["ixmax"] = df[["x2_sub", "x2_gt"]].min(axis=1)
#         df["iymin"] = df[["y1_sub", "y1_gt"]].max(axis=1)
#         df["iymax"] = df[["y2_sub", "y2_gt"]].min(axis=1)

#         df["iw"] = np.maximum(df["ixmax"] - df["ixmin"] + 1, 0.0)
#         df["ih"] = np.maximum(df["iymax"] - df["iymin"] + 1, 0.0)

#         # 2. calculate the area of inters
#         df["inters"] = df["iw"] * df["ih"]

#         # 3. calculate the area of union
#         df["uni"] = (
#             (df["x2_sub"] - df["x1_sub"] + 1) * (df["y2_sub"] - df["y1_sub"] + 1)
#             + (df["x2_gt"] - df["x1_gt"] + 1) * (df["y2_gt"] - df["y1_gt"] + 1)
#             - df["inters"]
#         )
#         # print(uni)
#         # 4. calculate the overlaps between pred_box and gt_box
#         df["iou"] = df["inters"] / df["uni"]

#         return df.drop(
#             ["ixmin", "ixmax", "iymin", "iymax", "iw", "ih", "inters", "uni"], axis=1
#         )

#     def filter_to_top_label_match(self, sub_labels):
#         """
#         Ensures ground truth boxes are only linked to the box
#         in the submission file with the highest IoU.
#         """
#         return (
#             sub_labels.sort_values("iou", ascending=False)
#             .groupby(["video_frame", "label_gt"])
#             .first()
#             .reset_index()
#         )

#     def add_isCorrect_col(self, sub_labels):
#         """
#         Adds True/False column if the ground truth label
#         and submission label are identical
#         """
#         sub_labels["isCorrect"] = (
#             sub_labels["label_gt"] == sub_labels["label_sub"]
#         ) & (sub_labels["iou"] >= self.iou_threshold)
#         return sub_labels

#     def calculate_metric_weighted(
#         self, sub_labels, weight_col="isDefinitiveImpact", weight=1000
#     ):
#         """
#         Calculates weighted accuracy score metric.
#         """
#         sub_labels["weight"] = sub_labels.apply(
#             lambda x: weight if x[weight_col] else 1, axis=1
#         )
#         y_pred = sub_labels["isCorrect"].values
#         y_true = np.ones_like(y_pred)
#         weight = sub_labels["weight"]
#         return accuracy_score(y_true, y_pred, sample_weight=weight)

#     def score(self, sub, labels_df=None, drop_extra_cols=True):
#         """
#         Scores the submission file against the labels.

#         Returns the evaluation metric score for the helmet
#         assignment kaggle competition.

#         If `check_constraints` is set to True, will return -999 if the
#             submission fails one of the submission constraints.
#         """
#         if labels_df is None:
#             labels_df = self.labels_df.copy()

#         if self.check_constraints:
#             if not self.check_submission(sub):
#                 return -999
#         sub_labels = self.merge_sub_labels(sub, labels_df, self.weight_col)
#         sub_labels = self.get_iou_df(sub_labels).copy()
#         sub_labels = self.filter_to_top_label_match(sub_labels).copy()
#         sub_labels = self.add_isCorrect_col(sub_labels)
#         score = self.calculate_metric_weighted(
#             sub_labels, self.weight_col, self.impact_weight
#         )
#         # Keep `sub_labels for review`
#         if drop_extra_cols:
#             drop_cols = [
#                 "x1_sub",
#                 "x2_sub",
#                 "y1_sub",
#                 "y2_sub",
#                 "x1_gt",
#                 "x2_gt",
#                 "y1_gt",
#                 "y2_gt",
#             ]
#             sub_labels = sub_labels.drop(drop_cols, axis=1)
#         self.sub_labels = sub_labels
#         return score

# if debug:
#     scorer = NFLAssignmentScorer(labels)
#     baseline_score = scorer.score(submission_df)
#     print(f"validation score {baseline_score:0.4f}") # this would be 0.33